# HW 1: Corporate Bond Pricing (due by 9.17 Tues)

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2019`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2019/HW1/HW1.ipynb`__  (Please use the same name for repository and ipynb file)
* Add solution code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [85]:
import numpy as np

In [86]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [87]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)

(10.0531143852866, 10.0)

In [92]:
# Put your code here to price the corporate bond

def corp_bond(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=10000):
    U = np.random.uniform(size=n_sample)
    default_time = -(1/def_rate)*np.log(U)
    price= recovery*np.exp(-rf_rate*default_time)
    price[default_time>mat] = np.exp(-rf_rate*mat)
    return price.mean()
  

# Call your function
corp_bond()

# Find the mean and std by calling the function 100 times. 
rep = [corp_bond() for i in range(100)]
print(np.mean(rep),np.std(rep))




0.9411257060169238 0.0012164588926014022


### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [89]:
# For example, antithetic method mean
n_sample =10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(10.0, 10.0)

In [91]:
# No include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=10000, antithetic=True, mean_match=True):
    U = np.random.uniform(size=n_sample)
    
    if(antithetic):
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
    else:
        default_time = -(1/def_rate)*np.log(U) 
        
    if(mean_match):
        default_time += 1/def_rate-default_time.mean()
        
    price= recovery*np.exp(-rf_rate*default_time)
    price[default_time>mat] = np.exp(-rf_rate*mat)
    return price.mean()
    


# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both

rep1 = [corp_bond_cv(antithetic=True, mean_match=False) for i in range(100)]
rep2 = [corp_bond_cv(antithetic=False, mean_match=True) for i in range(100)]
rep3 = [corp_bond_cv(antithetic=True, mean_match=True) for i in range(100)]
print(np.mean(rep1),np.std(rep1))
print(np.mean(rep2),np.std(rep2))
print(np.mean(rep3),np.std(rep3))


0.9411204468305248 0.0007980554625949568
0.9415309342658227 0.006613671523808137
0.941424032781611 0.0032851745974871566


### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?

In [110]:
### Put the analytic expression for the corporate bond price
mat=1
def_rate=0.03
rf_rate=0.04
recovery=0.3
analytic_price = recovery*(def_rate/(def_rate+rf_rate))*(1-np.exp(-(def_rate+rf_rate)*mat))+np.exp(-(def_rate+rf_rate)*mat)
analytic_price

0.9410860430608978